## Phase 4: V1 vs V2 Controller Comparison Validation

Test both controllers side-by-side with identical conditions to validate proper integration.

### Phase 4.1: Load V2 Controller Components

In [ ]:
def load_v2_controller_components():
    """Load V2 controller for side-by-side comparison with V1."""
    
    print("Loading V2 Controller Components")
    print("=" * 33)
    
    # Import V2 components
    from V2.robust_mpc.core import RobustMPCController
    from V2.robust_mpc.models import ProbabilisticTransformer
    from V2.robust_mpc.estimators import KalmanStateEstimator
    from V2.robust_mpc.optimizers import GeneticOptimizer
    from V2.robust_mpc.data_buffer import DataBuffer
    import yaml
    
    # Load V2 configuration
    config_path = Path("../../V2/config.yaml")
    with open(config_path, 'r') as f:
        v2_config = yaml.safe_load(f)
    
    print(f"✓ V2 configuration loaded from {config_path}")
    
    # Load V2 model
    v2_model_path = Path("../../V2/models/best_model.pth")
    if v2_model_path.exists():
        v2_model = load_trained_model(v2_model_path, device=DEVICE, validate=True)
        print(f"✓ V2 model loaded from {v2_model_path}")
    else:
        # Fallback to V1 model for comparison (same architecture)
        print(f"⚠️  V2 model not found, using V1 model for architecture comparison")
        v2_model = v1_model
    
    # Create V2 components
    estimator = KalmanStateEstimator(
        process_noise=v2_config['kalman']['process_noise'],
        measurement_noise=v2_config['kalman']['measurement_noise'],
        initial_uncertainty=v2_config['kalman']['initial_uncertainty']
    )
    
    optimizer = GeneticOptimizer(
        population_size=v2_config['genetic']['population_size'],
        generations=v2_config['genetic']['generations'],
        mutation_rate=v2_config['genetic']['mutation_rate'],
        crossover_rate=v2_config['genetic']['crossover_rate']
    )
    
    # Create V2 controller
    v2_controller = RobustMPCController(
        model=v2_model,
        estimator=estimator,
        optimizer=optimizer,
        config=v2_config,
        device=DEVICE,
        verbose=False  # Quiet for comparison
    )
    
    print(f"✓ V2 RobustMPCController created successfully")
    print(f"  Model device: {next(v2_model.parameters()).device}")
    print(f"  Estimator: {type(estimator).__name__}")
    print(f"  Optimizer: {type(optimizer).__name__}")
    
    return v2_controller, v2_config

# Load V2 controller components
try:
    v2_controller, v2_config = load_v2_controller_components()
    v2_loaded = True
except Exception as e:
    print(f"❌ Failed to load V2 controller: {e}")
    print(f"   Will proceed with V1-only validation")
    v2_loaded = False